# Конвейер обработки текста

## Настройка рабочей среды

In [1]:
!python -m spacy info


============================== Info about spaCy ==============================

spaCy version    3.7.2                         
Location         C:\prj\nlp_spacy\venv\lib\site-packages\spacy
Platform         Windows-10-10.0.22621-SP0     
Python version   3.10.8                        
Pipelines        en_core_web_sm (3.7.1), ru_core_news_sm (3.7.0)


[!] As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use
the full pipeline package name 'en_core_web_sm' instead.

In [0]:
#!python -m spacy download en

In [2]:
#!python -m spacy download en_core_web_sm

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     --------------------------------------- 0.1/12.8 MB 726.2 kB/s eta 0:00:18
     --------------------------------------- 0.1/12.8 MB 726.2 kB/s eta 0:00:18
      -------------------------------------- 0.2/12.8 MB 697.2 kB/s eta 0:00:19
      -------------------------------------- 0.2/12.8 MB 758.5 kB/s eta 0:00:17
      -------------------------------------- 0.3/12.8 MB 863.3 kB/s eta 0:00:15
      -------------------------------------- 0.3/12.8 MB 863.3 kB/s eta 0:00:15
      -------------------------------------- 0.3/12.8 MB 701.4 kB/s eta 0:00:18
     - ------------------------------------- 0.4/12.8 MB 836.4 kB/s eta 0:00:15
     - ------------------------------------- 0.4/12.8 MB 836.4 kB/s eta 


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


You can now load the package via spacy.load('en_core_web_sm')

In [3]:
# !python -m spacy download ru_core_news_sm

     ---------------------------------------- 0.0/15.3 MB ? eta -:--:--
     ---------------------------------------- 0.0/15.3 MB 1.4 MB/s eta 0:00:12
     ---------------------------------------- 0.1/15.3 MB 1.3 MB/s eta 0:00:12
     ---------------------------------------- 0.2/15.3 MB 1.1 MB/s eta 0:00:14
      --------------------------------------- 0.2/15.3 MB 1.6 MB/s eta 0:00:10
      --------------------------------------- 0.2/15.3 MB 1.6 MB/s eta 0:00:10
     - -------------------------------------- 0.5/15.3 MB 1.9 MB/s eta 0:00:08
     - -------------------------------------- 0.5/15.3 MB 1.9 MB/s eta 0:00:08
     - -------------------------------------- 0.5/15.3 MB 1.9 MB/s eta 0:00:08
     - -------------------------------------- 0.6/15.3 MB 1.4 MB/s eta 0:00:11
     -- ------------------------------------- 0.8/15.3 MB 1.8 MB/s eta 0:00:09
     -- ------------------------------------- 0.9/15.3 MB 1.8 MB/s eta 0:00:08
     -- ------------------------------------- 1.1/15.3 MB 1

You can now load the package via spacy.load('ru_core_news_sm')

## Загрузка корпуса

In [4]:
import spacy

In [21]:
nlp_en = spacy.load('en_core_web_sm')
nlp_ru = spacy.load('ru_core_news_sm')

## Токенизация

In [6]:
doc = nlp_en(u'I am flying to Frisco')
print([w.text for w in doc])

['I', 'am', 'flying', 'to', 'Frisco']


In [7]:
doc = nlp_ru(u'Я лечу в Казань')
print([w.text for w in doc])

['Я', 'лечу', 'в', 'Казань']


## Лемматизация

In [8]:
doc = nlp_en(u'this product integrates both libraries for downloading and applying patches')
for token in doc:
    print(token.text, token.lemma_)

this this
product product
integrates integrate
both both
libraries library
for for
downloading download
and and
applying apply
patches patch


In [9]:
#!pip install tabulate

In [10]:
from tabulate import tabulate

In [11]:
doc = nlp_ru(u'этот продукт объединяет обе библиотеки для загрузки и применения исправлений.')
print(tabulate([[token.text, token.lemma_] for token in doc], headers=['исх', 'лемма']))
# for token in doc:
#     print(token.text,'\t', token.lemma_)

исх          лемма
-----------  -----------
этот         этот
продукт      продукт
объединяет   объединять
обе          оба
библиотеки   библиотека
для          для
загрузки     загрузка
и            и
применения   применение
исправлений  исправление
.            .


## Использование лемматизации для распознавания смысла

In [18]:
from spacy.symbols import ORTH, LEMMA, NORM

In [25]:
doc = nlp_en(u'I am flying to Frisco')
print([w.text for w in doc])

nlp_en.get_pipe("attribute_ruler").add([[{"TEXT": "Frisco"}]], {"LEMMA": "San Francisco"})
# special_case = [{ORTH: u'Frisco', LEMMA: u'San Francisco'}]
# nlp_en.tokenizer.add_special_case(u'Frisco', special_case)
print([w.lemma_ for w in nlp_en(u'I am flying to Frisco')])

['I', 'am', 'flying', 'to', 'Frisco']
['I', 'be', 'fly', 'to', 'San Francisco']


In [27]:
doc = nlp_ru(u'Я лечу в Питер')
print([w.text for w in doc])

nlp_ru.get_pipe("attribute_ruler").add([[{"TEXT": "Питер"}]], {"LEMMA": "Санкт - Петербург"})
print([w.lemma_ for w in nlp_ru(u'Я лечу в Питер')])

['Я', 'лечу', 'в', 'Питер']
['я', 'лечу', 'в', 'Санкт - Петербург']


## Поиск соответствующих глаголов с помощью тегов частей речи

In [33]:
doc = nlp_en(u'I have flown to LA. Now I am flying to Frisco.')
print([w.text for w in doc if w.tag_== 'VBG' or w.tag_== 'VB'])

['flying']


In [34]:
print([w.tag_ for w in doc ])


['PRP', 'VBP', 'VBN', 'IN', 'NNP', '.', 'RB', 'PRP', 'VBP', 'VBG', 'IN', 'NNP', '.']


In [35]:
doc = nlp_ru(u'Я летал в Москву. Сейчас я лечу в Питер')
print([w.text for w in doc if w.tag_== 'VERB'])

['летал', 'лечу']


In [36]:
print([w.tag_ for w in doc ])


['PRON', 'VERB', 'ADP', 'PROPN', 'PUNCT', 'ADV', 'PRON', 'VERB', 'ADP', 'PROPN']


In [37]:
for token in doc:
    print(token.text, token.pos_, token.dep_)

Я PRON nsubj
летал VERB ROOT
в ADP case
Москву PROPN obl
. PUNCT punct
Сейчас ADV advmod
я PRON nsubj
лечу VERB ROOT
в ADP case
Питер PROPN obl


In [38]:
doc = nlp_ru(u'этот продукт объединяет обе библиотеки для загрузки и применения исправлений.')
print(tabulate([[token.text, token.pos_, token.dep_] for token in doc]))

-----------  -----  ----------
этот         DET    det
продукт      NOUN   nsubj
объединяет   VERB   ROOT
обе          NUM    nummod:gov
библиотеки   NOUN   obj
для          ADP    case
загрузки     NOUN   nmod
и            CCONJ  cc
применения   NOUN   conj
исправлений  NOUN   nmod
.            PUNCT  punct
-----------  -----  ----------


In [39]:
doc = nlp_ru(u'Я летал в Москву. Сейчас я лечу в Питер')
print(tabulate([[token.text, token.pos_, token.dep_] for token in doc]))

------  -----  ------
Я       PRON   nsubj
летал   VERB   ROOT
в       ADP    case
Москву  PROPN  obl
.       PUNCT  punct
Сейчас  ADV    advmod
я       PRON   nsubj
лечу    VERB   ROOT
в       ADP    case
Питер   PROPN  obl
------  -----  ------
